In [1]:
import graphlab as gl
import re

A newer version of GraphLab Create (v1.9) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
import requests
import json

In [3]:
from __future__ import division

In [11]:
listOfRestauranIds = []
with open('restaurantMapping.json') as json_data:
    loadingResJsons = json.load(json_data)
print loadingResJsons['61178']
print type(loadingResJsons)
count = 0

urlList = loadingResJsons.keys()    
print len(urlList), int(urlList[0]), urlList[1]

Rolls On Wheels, Banaswadi
<type 'dict'>
7730 60417 55292


In [12]:
urlList = [int(i) for i in urlList]
print len(urlList)
urlList = sorted(list(urlList))

7730


In [234]:
urlList = urlList[6000:]

In [6]:
import time
import json

In [235]:
res = open("resTryFinal.csv", "w")
headers = {
                'X-Zomato-API-Key': "7749b19667964b87a3efc739e254ada2",
                'cache-control': "no-cache",
                "user-agent" : "myapp",
                'postman-token': "58b1afdb-8a80-b361-bfc8-4949a8215857"
        }
url = "https://api.zomato.com/v1/restaurant.json/56126"
#urlList = loadingResJsons.keys()
fpJSONFull = open('dataJsonFullFinal.txt', 'w')
lol4 = []
print len(urlList)
TimeOut4 = []
triedTwice4 = []
triedThrice4 = []
exCount4 = 0
fpExcept = open('unVisited4.txt', 'w')
fpJSON = open("dataJsonFinal.txt",'w')
count = 0
headerForCsv = "id" + "," + "name" + "," + "cuisines" + "," + "avgCostForTwo" + "," + "takeaway_flag" + "," + "smoking_area_flag"+ "," + "isPureVeg" + "," + "hasBar" + "," + "hasDineIn" + "," + "hasDelivery" + "," + "hasAC" + "," + "hasWifi" + "," + "hasLiveMusic" + "," + "acceptsCreditCards" + "," + "buffet_details" + "," + "numOfUserReviews" + "," + "AverageRatings" + "," + "locality" + "," + "startTime" + ","+ "endTime"+","+ "editorFood" + "," + "editorAmbience" + "," + "editorOverall" + "," + "editorService"
res.write(headerForCsv)
res.write('\n')
noWorkIds = []

#print urlList
while(urlList):
    urlList = sorted(urlList)
    u = urlList.pop(0)
    #print u
    #print str(u)
    count = count + 1
    if((count % 25) == 0):
        print "count is", count, "left : ", len(urlList)
        time.sleep(1)
        
    try:    
        currentURL = "https://api.zomato.com/v1/restaurant.json/" + str(u)
        response = requests.get(currentURL, headers=headers,timeout=3.5)
        #print type(response.content)
        #print type(response.json())
        d = response.json()
        jsonKeys = ['name','cuisines','avgCostForTwo','takeaway_flag','smoking_area_flag', 'isPureVeg', 'hasBar', 'hasDineIn', 'hasDelivery','hasAC','hasWifi','hasLiveMusic','acceptsCreditCards','buffet_details','timings']
        wanted_keys = ['name','cuisines','avgCostForTwo','takeaway_flag','smoking_area_flag', 'isPureVeg', 'hasBar', 'hasDineIn', 'hasDelivery','hasAC','hasWifi','hasLiveMusic','acceptsCreditCards','buffet_details']
        writeJson = str(u)

        fpJSONFull.write(json.dumps(d,indent=True))
        fpJSONFull.write("\n")
        b = {key:d[key] for key in jsonKeys}
        b['id'] = u
        b['latitude'] = d['location']['latitude']
        b['longitude'] = d['location']['longitude']
        fpJSON.write(json.dumps(b,indent=True))
        fpJSON.write("\n")
        
        for each in wanted_keys :
            if(str(d[each]) != ""):
                ch = re.sub(""",""","#",str(d[each]),re.I|re.S)
                writeJson = writeJson +","+ ch
                #print each, str(d[each])
            else:
                writeJson = writeJson +","+ "N/A"
                #print each, 'NA'
            #print d[each]
        #print "reached Here"
        userReviews = d['userReviews']
        #print "reached Here2"

        numberOfReviews = re.search("""u'count':\s*(.*?),""",str(userReviews),re.I|re.S).group(1)
        #print numberOfReviews
        writeJson = writeJson + "," + numberOfReviews
        userReviews = str(userReviews)
        userRatings = re.findall("""'rating':\s*(.*?),""", str(userReviews) ,re.I|re.S)
        #print userRatings
        avgRating = 0
        for rate in userRatings:
            avgRating = avgRating + int(rate)
        if(len(userRatings)!=0):
            avgRating = avgRating/len(userRatings)
        
        writeJson = writeJson +","+  str(avgRating)
        #print avgRating
        
        if(str(d['location']) != ""):
            locate = d['location']
            loc = locate['locality']
            loc = re.sub(""",""","#",loc,re.I|re.S)
            writeJson = writeJson +","+  loc
            #print "locality", loc
        else:
            writeJson = writeJson +","+  "N/A"
            #print "locality", 'N/A'
        #print "here"
        if(str(d['timings']) != ""):
            tim = d['timings']
        #    writeJson = writeJson +","+  t
            startTime = re.search("""\s*(.*?)\s*AM""",tim,re.I|re.S)
            if(startTime != None):
                startTime = startTime.group(1)
                startTime = re.sub(""",""", "#",startTime,re.I|re.S)
            else:
                startTime = 'N/A'
            
            endTime = re.search("""to\s*(.*?)\s*PM""",tim,re.I|re.S)
            if(endTime != None):
                endTime = endTime.group(1)
                endTime = re.sub(""",""", "#",endTime,re.I|re.S)
                #endTime = str(int(endTime) + 12
            else:
                endTime = 'N/A'
            #print "endTime", endTime
            writeJson = writeJson +","+ startTime + "," + endTime
            
        else:
            writeJson = writeJson +","+  "N/A" + "," + "N/A"
            #print "tim", 'N/A'
        #print "here2"
        editor = ""
        editorAmbience = 'N/A'
        editorFood = 'N/A'
        editorOverall = 'N/A'
        editorService = 'N/A'
        
        #print 'reached Here3'
        editor = re.search("""ambience\s*:\s*(.*?),""", str(d), re.I|re.S)
        if(editor != None):
            editorF = re.search("""food':\s*(.*?),""", str(d['editorRating']), re.I|re.S)
            if(editorF != None):
                editorFood = editorF.group(1)
            
            editorAmb = re.search("""ambience':\s*(.*?),""", str(d['editorRating']), re.I|re.S)
            if(editorAmb != None):
                editorAmbience = editorAmb.group(1)

            editorO = re.search("""overall':\s*(.*?),""", str(d['editorRating']), re.I|re.S)
            if(editorO != None):
                editorOverall = editorO.group(1)
            editorServ = re.search("""service':\s*(.*?),""", str(d['editorRating']), re.I|re.S)
            if(editorServ != None):
                editorService = editorServ.group(1)

        writeJson = writeJson +","+ editorFood + "," + editorAmbience +","+ editorOverall +","+ editorService
        
        res.write(writeJson)
        res.write('\n')
        
    except Exception,e:
     
        print repr(e)
        exCount4 = exCount4 + 1
         
        if("time" in repr(e)):
            if(u not in triedTwice4):
                urlList.append(u)
                triedTwice4.append(u)
            elif(u not in triedThrice4):
                urlList.append(u)
                triedThrice4.append(u)
            else:
                TimeOut4.append(u)       
#                fpExcept.write(str(u))
#                fpExcept.write("\n")
    
        else:
            lol4.append(u)
#            fpExcept.write(str(u))
#            fpExcept.write("\n")
            
        print "except entered", exCount4
fpExcept.close()
res.close()

1730
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 1
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 2
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 3
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 4
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 5
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 6
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 7
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 8
count is 25 left :  1713
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 9
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except 

In [13]:
headers = {
                'X-Zomato-API-Key': "7749b19667964b87a3efc739e254ada2",
                'cache-control': "no-cache",
                "user-agent" : "myapp",
                'postman-token': "58b1afdb-8a80-b361-bfc8-4949a8215857"
        }

fpJSONFull = open('dataJsonFullFinal.txt', 'w')
lol4 = []
print len(urlList)
TimeOut4 = []
triedTwice4 = []
triedThrice4 = []
exCount4 = 0
fpExcept = open('unVisited4.txt', 'w')
fpJSON = open("dataJsonFinal.txt",'w')
count = 0

while(urlList):
    urlList = sorted(urlList)
    u = urlList.pop(0)

    count = count + 1
    if((count % 50) == 0):
        print "count is", count, "left : ", len(urlList)
        time.sleep(1)
        
    try:    
        currentURL = "https://api.zomato.com/v1/restaurant.json/" + str(u)
        response = requests.get(currentURL, headers=headers,timeout=3.5)
        #print type(response.content)
        #print type(response.json())
        d = response.json()
        jsonKeys = ['name','cuisines','avgCostForTwo','takeaway_flag','smoking_area_flag', 'isPureVeg', 'hasBar', 'hasDineIn', 'hasDelivery','hasAC','hasWifi','hasLiveMusic','acceptsCreditCards','buffet_details','timings']

        fpJSONFull.write(json.dumps(d,indent=True))
        fpJSONFull.write("\n")
        b = {key:d[key] for key in jsonKeys}
        b['id'] = u
        b['latitude'] = d['location']['latitude']
        b['longitude'] = d['location']['longitude']
        fpJSON.write(json.dumps(b,indent=True))
        fpJSON.write("\n")
        
    except Exception,e:
     
        print repr(e)
        exCount4 = exCount4 + 1
         
        if("time" in repr(e)):
            if(u not in triedTwice4):
                urlList.append(u)
                triedTwice4.append(u)
            elif(u not in triedThrice4):
                urlList.append(u)
                triedThrice4.append(u)
            else:
                TimeOut4.append(u)       
#                fpExcept.write(str(u))
#                fpExcept.write("\n")
    
        else:
            lol4.append(u)
#            fpExcept.write(str(u))
#            fpExcept.write("\n")
            
        print "except entered", exCount4
fpExcept.close()

7730
count is 50 left :  7680
count is 100 left :  7630
count is 150 left :  7580
count is 200 left :  7530
count is 250 left :  7480
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 1
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 2
count is 300 left :  7432
count is 350 left :  7382
count is 400 left :  7332
count is 450 left :  7282
count is 500 left :  7232
count is 550 left :  7182
count is 600 left :  7132
count is 650 left :  7082
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 3
count is 700 left :  7033
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 4
count is 750 left :  6984
SSLError(SSLError(SSLError('_ssl.c:574: The handshake operation timed out',),),)
except entered 5
count is 800 left :  6935
count is 850 left :  6885
count is 900 left :  6835
count is 950 left :  6785
SSLError(SSL

NameError: name 'res' is not defined

In [14]:
fpJSON.close()

In [46]:
res = open("try.csv", "w")
str1 = "helo"
str2 = "heaslo"
st = str1 + "," + "str2"
res.write(st)

res.write('\n')

str1 = "helo1"
str2 = "heaslo2"
st = str1 + "," + "str2"
res.write(st)
res.close()


In [236]:
t = gl.SFrame.read_csv("resTry.csv")
t1 = gl.SFrame.read_csv("resTry1,csv")

PROGRESS: Finished parsing file C:\Users\Tarun Tater\Desktop\sem8\ML2Project\resTry.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.442965 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[long,str,str,long,long,long,long,long,long,long,long,long,long,long,str,long,float,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Unable to parse line "52500,Tomatina,Continental# Chinese# North Indian,800,1,1,1,N/A,N/A,N/A,,,,,,,,,,,,,,"
PROGRESS: Unable to parse line "On Other Days TDH Service (Se,213,3.6,HSR,10:30,3:30,N/A,0,1,1,1,0,2,1,N/A,19,4,JP Nagar,7,11,N/A,N/A,N/A,N/A"
PROGRESS: 2 lines failed to parse correctly
PROGRESS: Finished parsing file C:\Users\Tarun Tater\Desktop\sem8\ML2Project\resTry.csv
PROGRESS: Parsing complete

RuntimeError: Runtime Exception. No files corresponding to the specified path (C:\Users\Tarun Tater\Desktop\sem8\ML2Project\resTry1,csv).

In [237]:
print t

+-------+--------------------------------+-------------------------------+---------------+
|   id  |              name              |            cuisines           | avgCostForTwo |
+-------+--------------------------------+-------------------------------+---------------+
| 38764 |         Yana Sizzlers          | Chinese# Thai# Seafood# It... |      1100     |
| 50000 |          Spicy North           |     North Indian# Chinese     |      300      |
| 50001 |              1947              |     North Indian# Chinese     |      750      |
| 50003 | 100 FT Boutique Bar Restaurant | Continental# Mediterranean... |      1500     |
|  None |                                |                               |      None     |
| 50004 |   12th Main - Grand Mercure    |        European# Asian        |      2000     |
| 50005 |         18 China Town          |            Chinese            |      550      |
| 50008 | 9th Mile Dhaba - Royal Orc...  |          North Indian         |      1200     |

In [120]:
for key in d:
    print key

reservation_text
takeaway_flag
cuisines
opening_soon_flag
nightlife_flag
establishment_types
is_open_now
has_emirates_discount
image_150_150
image_310_150
userRating
emirates_discount_flag
bar_wineandbeer_flag
menu
sports_bar_flag
entry_fee
id
outdoor_seating_flag
hasWifi
bar_byob_flag
group
hasBar
cost_of_beer
serves_veg_flag
serviceCharge
bar_fullbar_flag
discounts
editorRating
sheesha_flag
location
stag_entry_flag
isPureVeg
mobile_phone_display
buffet_details
hasAC
vat
citibank_discount_detail
mobile_phone
bar_wineonly_flag
happy_hour_details
status_id
hasDelivery
phone
acceptsCreditCards
image_470_310
hasDineIn
halal_flag
serves_nonveg_flag
userReviews
name
photos
url
editorReview
timings
smoking_area_flag
has_citibank_discount
image_310_310
meal_coupon_flag
avgCostForTwo
hasLiveMusic


In [121]:
print d

{u'reservation_text': u'', u'takeaway_flag': 1, u'cuisines': u'South Indian', u'opening_soon_flag': 0, u'nightlife_flag': 0, u'establishment_types': [{u'establishment_type': {u'id': 16, u'name': u'Casual Dining'}}], u'is_open_now': 0, u'has_emirates_discount': 0, u'image_150_150': u'https://b.zmtcdn.com/images/default_thumb_150_150.png', u'image_310_150': u'https://b.zmtcdn.com/images/default_thumb_310_150.png', u'userRating': {u'aggregate': u'0', u'votes': 3, u'rating_text': u'Not rated', u'rating_color': u'CBCBC8'}, u'emirates_discount_flag': u'', u'bar_wineandbeer_flag': 0, u'menu': [], u'sports_bar_flag': 0, u'entry_fee': u'0', u'id': 18089689, u'outdoor_seating_flag': 2, u'hasWifi': 0, u'bar_byob_flag': 0, u'group': [], u'hasBar': 0, u'cost_of_beer': 0, u'serves_veg_flag': 0, u'serviceCharge': 0, u'bar_fullbar_flag': 0, u'discounts': [], u'editorRating': {u'food': 0, u'ambience': 0, u'overall': 0, u'service': 0}, u'sheesha_flag': 0, u'location': {u'city': u'Bangalore', u'locality'

In [196]:
b = {key:d[key] for key in wanted_keys }

In [197]:
print type(b)

<type 'dict'>


In [198]:
b['id'] = 231

********************************Code For Training**************************************

In [239]:
t = gl.SFrame.read_csv("resTry.csv")

PROGRESS: Finished parsing file C:\Users\Tarun Tater\Desktop\sem8\ML2Project\resTry.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.063041 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[long,str,str,long,long,long,long,long,long,long,long,long,long,long,str,long,float,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Unable to parse line "52500,Tomatina,Continental# Chinese# North Indian,800,1,1,1,N/A,N/A,N/A,,,,,,,,,,,,,,"
PROGRESS: Unable to parse line "On Other Days TDH Service (Se,213,3.6,HSR,10:30,3:30,N/A,0,1,1,1,0,2,1,N/A,19,4,JP Nagar,7,11,N/A,N/A,N/A,N/A"
PROGRESS: 2 lines failed to parse correctly
PROGRESS: Finished parsing file C:\Users\Tarun Tater\Desktop\sem8\ML2Project\resTry.csv
PROGRESS: Parsing complete

In [240]:
t1 = gl.SFrame.read_csv("resTry1.csv")

PROGRESS: Finished parsing file C:\Users\Tarun Tater\Desktop\sem8\ML2Project\resTry1.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.06004 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[long,str,str,long,long,long,long,long,long,long,long,long,long,long,str,long,float,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file C:\Users\Tarun Tater\Desktop\sem8\ML2Project\resTry1.csv
PROGRESS: Parsing completed. Parsed 1494 lines in 0.06304 secs.


In [241]:
t2 = gl.SFrame.read_csv("resTry2.csv")

PROGRESS: Finished parsing file C:\Users\Tarun Tater\Desktop\sem8\ML2Project\resTry2.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.088058 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[long,str,str,long,long,long,long,long,long,long,long,long,long,long,str,long,float,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file C:\Users\Tarun Tater\Desktop\sem8\ML2Project\resTry2.csv
PROGRESS: Parsing completed. Parsed 1500 lines in 0.121079 secs.


In [242]:
t3 = gl.SFrame.read_csv("resTry3.csv")

PROGRESS: Finished parsing file C:\Users\Tarun Tater\Desktop\sem8\ML2Project\resTry3.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.17511 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[long,str,str,long,long,long,long,long,long,long,long,long,long,long,str,long,float,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file C:\Users\Tarun Tater\Desktop\sem8\ML2Project\resTry3.csv
PROGRESS: Parsing completed. Parsed 1499 lines in 0.133087 secs.


In [243]:
import graphlab.aggregate as agg

In [244]:
from graphlab import SArray

In [ ]:
trainCheck['amountCategory'].sketch_summary()
trainFeatures.column_names

In [ ]:
trainFeaturesChange = gl.load_sframe('./trainFeaturesUpdated')
trainFeaturesChange.remove_column('offerdate')
numberOfTransactions = SArray(numberOfTransactions)
testFeaturesChange.add_column(numberOfTransactions)
brandLoyalty = trainFeaturesChange['numBrand']/trainFeaturesChange['numberOfTransactions'] 
trainFeaturesChange.add_column(brandLoyalty, 'brandLoyalty')
testFeaturesChange.rename({'X44' : 'numberOfTransactions'})
hasNotBoughtAny180 = -1*((trainCheck['company180'] == 0)*(trainCheck['category180'] == 0)*(trainCheck['brand180'] == 0))
copy = hasNotBoughtAny180[:]
testCheck1.add_columns([hasNotBoughtAny180,hasBoughtAll15],['hasNotBoughtAny180','hasBoughtAll15'])
hasNeverBoughtCompany = -1*((testFeaturesChange['numCompany'] == 0))

In [ ]:
model21= gl.random_forest_classifier.create(trainCheck1,target = 'repeater', verbose = True, class_weights = 'auto')
predictions = model21.classify(testCheck1)
upload = testCheck1['id', 'numBrand']
upload.remove_column('numBrand')
upload.add_column(predictions['probability'], 'repeatProbability')
upload.export_csv('submit21.csv', delimiter = ',', line_terminator = '\n', header = True, quote_level=2, quote_char = '"', na_rep = 'NA', file_header = '', file_footer = '', line_prefix = '', _no_prefix_on_first_value=False)

In [ ]:
model = gl.logistic_classifier.create(trainCheck1, target = 'repeater', max_iterations = 20)
predictions = model.predict(testCheck1, output_type='probability')
repeatProbability = gl.SArray(predictions)
upload = testFeaturesChange['id', 'numBrand']
upload.remove_column('numBrand')
upload.add_column(repeatProbability, 'repeatProbability')
upload.export_csv('submitlg2.csv', delimiter = ',', line_terminator = '\n', header = True, quote_level=2, quote_char = '"', na_rep = 'NA', file_header = '', file_footer = '', line_prefix = '', _no_prefix_on_first_value=False)

In [ ]:
testFeaturesChange.save("testFeaturesUpdated")
model15.save('m15')

In [250]:
t = t.append(t1)

In [254]:
t = t.append(t2)

In [255]:
t = t.append(t3)

In [256]:
t.num_rows

<bound method SFrame.num_rows of Columns:
	id	int
	name	str
	cuisines	str
	avgCostForTwo	int
	takeaway_flag	int
	smoking_area_flag	int
	isPureVeg	int
	hasBar	int
	hasDineIn	int
	hasDelivery	int
	hasAC	int
	hasWifi	int
	hasLiveMusic	int
	acceptsCreditCards	int
	buffet_details	str
	numOfUserReviews	int
	AverageRatings	float
	locality	str
	startTime	str
	endTime	str
	editorFood	str
	editorAmbience	str
	editorOverall	str
	editorService	str

Rows: 5977

Data:
+-------+--------------------------------+-------------------------------+---------------+
|   id  |              name              |            cuisines           | avgCostForTwo |
+-------+--------------------------------+-------------------------------+---------------+
| 38764 |         Yana Sizzlers          | Chinese# Thai# Seafood# It... |      1100     |
| 50000 |          Spicy North           |     North Indian# Chinese     |      300      |
| 50001 |              1947              |     North Indian# Chinese     |      750   

In [257]:
t = t.remove_column('buffet_details')

In [273]:
with open("dishReviews.json") as json_file:
    json_data = json.load(json_file)
    

In [275]:
print json_data["The Rasaganga, Bellandur"]

[[u'sambar', 0.8410714285714286, [u'sambar also good', u'sambar good and fresh', u'sambar slightly spicier', u'crisp yummy scrumptious sambar']], [u'fried rice', 0.825, [u'fried rice regular', u'biryani fried rice', u'fried rice', u'fried rice decent']], [u'chaat', 0.8214285714285715, [u'chaats also good', u'dahi chaat better', u'dahi potato chaat tasted almost similar']], [u'idli', 0.7875, [u'vadas and rawa idlies good', u'usual things used idli', u'fluffy idlis', u'big idlies']], [u'coffee', 0.75, [u'strong coffee cost', u'coffee really good', u'coffee good', u'good coffee']], [u'rice', 0.7482142857142857, [u'prices also reasonable', u'prices also reasonable', u'less prices', u'fried rice regular', u'food priced low ac fine', u'biryani fried rice', u'better expectation prices tad higher', u'prices lower', u'fried rice', u'jeera rice plenty times and good', u'decent affordable price', u'2xroti curd rice pulav soup dal', u'fried rice decent', u'several times prices', u'good prices okay

In [276]:
t['numOfUserReviews'].sketch_summary()


+--------------------+---------------+----------+
|        item        |     value     | is exact |
+--------------------+---------------+----------+
|       Length       |      5977     |   Yes    |
|        Min         |      0.0      |   Yes    |
|        Max         |     3460.0    |   Yes    |
|        Mean        |  62.690041841 |   Yes    |
|        Sum         |    374573.0   |   Yes    |
|      Variance      | 25399.6117084 |   Yes    |
| Standard Deviation | 159.372556321 |   Yes    |
|  # Missing Values  |       2       |   Yes    |
|  # unique values   |      504      |    No    |
+--------------------+---------------+----------+

Most frequent items:
+-------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| value |  0  |  1  |  2  |  3  |  5  |  4  |  6  |  9  |  7  |  8  |
+-------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| count | 446 | 385 | 330 | 285 | 232 | 210 | 165 | 152 | 135 | 133 |
+-------+-----+-----+-----+-----+-----+-----+--

In [278]:
 with open("dataJsonFull.txt") as d1:
    j1 = json.loads(d1)
    

TypeError: expected string or buffer

%%%%%%%%%%%%%%%%%%%%%%%%%%TIMINGS%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [279]:
timingsList = ["12:30 PM to 3:30 PM, 7 PM to 11 PM", "11 AM to 11 PM", "10:30 AM to 11 PM", "12 Noon to 11 PM (Mon-Thu), 12 Noon to 1 AM (Fri-Sun)",
              "9 AM to 11 PM", "12 Noon to 11 PM", "11 AM to 11:30 PM", "12:30 PM to 11 PM", "12 Noon to 3 PM, 7 PM to 10:15 PM",
              "12 Noon to 11 PM (Tue-Thu, Sat-Sun), 1:30 PM to 11 PM (Fri), Mon Closed", ""]

In [283]:
a = "asd"
print a[-2:]

sd


In [336]:
def hoursAndMinsCalculate(startTime):
    if(':' in startTime):
#        print startTime
        hoursAndMins = re.search("""(.*?):(.*)""",startTime, re.I|re.S)
#        print "ya", hoursAndMins.group(1), type(hoursAndMins.group(1)), len(hoursAndMins.group(1))
        hours = hoursAndMins.group(1)
        mins = hoursAndMins.group(2)
        timings = str(int(hours)*100 + int(mins))
    else:
        timings = str(int(startTime)*100)
#    print "yup", timings
    return timings

In [351]:
timingsList = ["12 Noon to 11 PM (Tue-Thu, Sat-Sun), 1:30 PM to 11 PM (Fri), Mon Closed",
              "12:30 PM to 3:30 PM, 7 PM to 11 PM", "12 Noon to 11 PM", "11 AM to 11:30 PM", "12:30 PM to 11 PM",
              "12 Noon to 11 PM",]

In [394]:
def timingsCalculate(eachTime):
    timings = ""
    print eachTime
    startTime = re.search("""(.*?)\sto""",eachTime,re.I|re.S).group(1)
    checkAMPM = startTime[-2:]
    if(checkAMPM == "AM"):
        startTime = re.search("""(.*?)\sAM""",startTime, re.I|re.S).group(1)
        timings = hoursAndMinsCalculate(startTime) + "-"
    elif(checkAMPM == "PM"):
        startTime = re.search("""(.*?)\sPM""",startTime, re.I|re.S).group(1)
        tom = hoursAndMinsCalculate(startTime) 
        if( "12" not in tom ):
            timings = str(int(tom) + 1200) + "-"
        else:
            timings = tom + "-"
    elif(checkAMPM == 'on'):
        timings = "1200-"

    endTime = re.search("""to\s(.*)""",eachTime,re.I|re.S).group(1)
    checkAMPM = endTime[-2:]
    if(checkAMPM == "AM"):
        endTime = re.search("""(.*?)\sAM""",endTime, re.I|re.S).group(1)
        timings = timings + str(int(hoursAndMinsCalculate(endTime)) + 2400)
    elif(checkAMPM == "PM"):
        endTime = re.search("""(.*?)\sPM""",endTime, re.I|re.S).group(1)
        timings = timings + str(int(hoursAndMinsCalculate(endTime)) + 1200)

    return timings

In [414]:
timingsList = ["12:30 PM to 3:30 PM (Mon-Sat), Sun Closed ", "12 Noon to 11 PM (Tue-Thu, Sat-Sun), 1:30 PM to 11 PM (Fri), Monday Closed",
              "12:30 PM to 3:30 PM, 7 PM to 11 PM"]

In [417]:
timingsList = ["12:30 PM to 3:30 PM (Mon-Sat), Sun Closed "]

In [415]:
def checkClosedDay(whichDay):
    if((whichDay[-5:-3]) == "on"):
        return 1
    elif((whichDay[-6:-3]) == "ues"):
        return 2
    elif((whichDay[-6:-3]) == "nes"):
        return 3
    elif((whichDay[-6:-3]) == "urs"):
        return 4
    elif((whichDay[-5:-3]) == "ri"):
        return 5
    elif((whichDay[-5:-3]) == "ur"):
        return 6
    else:
        return 7

In [418]:
for eachTime in timingsList : 
    #print eachTime
    detectComma = re.findall("""to""",eachTime,re.I|re.S) #more than one time present
    numTimes = len(detectComma)
    timings = ""
    if(numTimes == 1):
        timings = timingsCalculate(eachTime)
    if((numTimes == 2) and ("(" not in eachTime)):
        listOfTimings = eachTime.split(',')
        timings = timingsCalculate(listOfTimings[0]) + '#'
        timings = timings + timingsCalculate(listOfTimings[1])
    
    #mon - 1, tue - 2, wed - 3, thurs - 4, fri -5, sat - 6
    if("Closed" in eachTime):
        
        ti = re.search("""(.*?)\s\(""",eachTime,re.I|re.S).group(1)
        detectComma = re.findall("""to""",ti,re.I|re.S) #more than one time present
        numTimes = len(detectComma)
        #11 AM to 11 PM (Mon-Sat), Sun Closed
        if(numTimes == 1):
            timings = timingsCalculate(ti)
        if(numTimes == 2):
            listOfTimings = ti.split(',')
            timings = timingsCalculate(listOfTimings[0]) + '#'
            timings = timings + timingsCalculate(listOfTimings[1])
    
        whichDay = re.search("""(.*?)\sClosed""", eachTime, re.I|re.S).group(1)
        print whichDay
        if(whichDay[-4] == " "):
            if(whichDay[-6] == ")"):
                whichDayClosed = whichDay[-3:]
                print whichDayClosed
            else:
                #multiples days closed
                asd = re.search("""\\),(.*?)""", whichDay, re.I|re.S).group(1)
                print asd, "asdsadasd"
        else:
            print checkClosedDay(whichDay)

12:30 PM to 3:30 PM (Mon-Sat), Sun Closed 
12:30 PM to 3:30 PM
12:30 PM to 3:30 PM (Mon-Sat), Sun
Sun


In [400]:
for eachTime in timingsList : 
    #print eachTime
    detectComma = re.findall("""to""",eachTime,re.I|re.S) #more than one time present
    #print detectComma
    numTimes = len(detectComma)
    timings = ""
    if(numTimes == 1):
        timings = timingsCalculate(eachTime)
    if((numTimes == 2) and ("(" not in eachTime)):
        listOfTimings = eachTime.split(',')
        timings = timingsCalculate(listOfTimings[0]) + '#'
        timings = timings + timingsCalculate(listOfTimings[1])
    
    #mon - 1, tue - 2, wed - 3, thurs - 4, fri -5, sat - 6
    if("Closed" in eachTime):
        
        ti = re.search("""(.*?)\s\(""",eachTime,re.I|re.S).group(1)
        detectComma = re.findall("""to""",ti,re.I|re.S) #more than one time present
        numTimes = len(detectComma)
        if(numTimes == 1):
            timings = timingsCalculate(ti)
        if(numTimes == 2):
            listOfTimings = ti.split(',')
            timings = timingsCalculate(listOfTimings[0]) + '#'
            timings = timings + timingsCalculate(listOfTimings[1])
    print eachTime, " is " , timings
        #whichDay = re.search("""(.*?)\sClosed""", eachTime, re.I|re.S).group(1)
        #print whichDay
        #if(whichDay[-4] == " "):
        #    if(whichDay[-6] == ")"):
        #        whichDayClosed = whichDay[-3:]
        #        print whichDayClosed
        #    else:
        #        #multiples days closed
        #        asd = re.search("""\\),(.*?)""", whichDay, re.I|re.S).group(1)
        #        print asd, "asdsadasd"
        #else:
        #    print checkClosedDay(whichDay)

12 Noon to 11 PM
12 Noon to 11 PM (Tue-Thu, Sat-Sun), 1:30 PM to 11 PM (Fri), Monday Closed  is  1200-2300
12:30 PM to 3:30 PM
 7 PM to 11 PM
12:30 PM to 3:30 PM, 7 PM to 11 PM  is  1230-1530#1900-2300


%%%%%%%%%%%%%%% Distance Between Two Places %%%%%%%%%%%%%%%

In [360]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [ ]:
#"Adda - The Restaurant"
#medi

In [362]:
print haversine(77.6174, 12.9226, 77.56944, 12.97833)

8.08783120658


In [413]:
with open("final_data.json","r") as f:
    data = json.load(f)

d = {}
i = 0
for restaurant in data:
    reviews = restaurant["userReviews"]
    dishes = restaurant["dishes"]
    i = i + 1
    if((i==10) or (i==20) or (i==2) or (i==4)):
        print i
    if((i%200) == 0):
        print i
    for review in reviews:
        userReview = review["reviewText"]
        dishForUser = {}
        for dish in dishes.keys():
            for eachDishReview in dish:
                if(eachDishReview in userReview):
                    try:
                        d[review["user_id"]][dish].append(eachDishReview)
                    except:
                        d[review["user_id"]][dish]=[]
                        d[review["user_id"]][dish].append(eachDishReview)
#import operator
#sorted_x = sorted(d.items(), key=operator.itemgetter(1))
#print sorted_x[-10000:-9990]
#print sum(d.values())/len(d.keys())

Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored


KeyboardInterrupt: 

In [ ]:
a = ""
print a